In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from pytorch_tabnet.tab_model import TabNetClassifier
from torch.optim import AdamW, lr_scheduler

In [ ]:
df = pd.read_csv('Structural data_coarse-grained.csv')
y = df['AMI'].values
X = df.drop(['AMI'], axis=1).values

In [ ]:
#Validatoin A, B, C
#validation A
selected_features = []
#validation B
#selected_features = [ 1,2,4,5,6,7,9,10,11,12, 17,19,20,21,22,23,24,25,26,27,28,30,31,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50] #验证C
#validation C
#selected_features = [29,51,14,32,13,15,16,18,8,3,0,52] 
total_features=np.arange(54).reshape((1, 54))
counter_features=np.setdiff1d(total_features,selected_features)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, log_loss
import numpy as np
NUM_SPLITS = 10
skf = StratifiedKFold(shuffle=True, random_state=79, n_splits=NUM_SPLITS)
metrics_per_fold = []
praf = ['precision', 'recall', 'acc', 'f1', 'specificity', 'log_loss']
for fold, (train_idx, val_idx) in enumerate(skf.split(X=X, y=y)):
    X_train_modified = X[train_idx].copy()
    X_val_modified = X[val_idx].copy().astype(float)
    X_val_modified[:, selected_features] = np.nan  # 不进行填充
    X_val_modified[np.isnan(X_val_modified)] = np.nanmean(X_val_modified)
    clf = TabNetClassifier()
    clf.fit(
        X_train=X_train_modified,
        y_train=y[train_idx],
        eval_set=[(X_val_modified, y[val_idx])],
        eval_metric=['logloss'],
        patience=16,
        max_epochs=100,
        batch_size=128
    )
    pred_proba = clf.predict_proba(X_val_modified)
    pred_label = clf.predict(X_val_modified)
    y_val = y[val_idx]
    current_logloss = log_loss(y_val, pred_proba)
    precision = precision_score(y_val, pred_label)
    recall_val = recall_score(y_val, pred_label)
    acc = accuracy_score(y_val, pred_label)
    f1 = f1_score(y_val, pred_label)
    specificity = recall_score(y_val, pred_label, pos_label=0)
    fold_metrics = [precision, recall_val, acc, f1, specificity, current_logloss]
    metrics_per_fold.append(fold_metrics)    
    print(f"\nFold {fold + 1} Performance:")
    for name, value in zip(praf, fold_metrics):
        print(f"{name}: {value:.4f}")

metrics_per_fold = np.array(metrics_per_fold)
print("\nFinal Metrics (Mean ± Std):")
for name, mean, std in zip(praf, np.mean(metrics_per_fold, axis=0), np.std(metrics_per_fold, axis=0, ddof=1)):
    print(f"{name}: {mean:.4f} ± {std:.4f}")
